In [ ]:
#load neccesary packages
using Eirene
using CSV
using DataFrames
using Plots
using RCall
using Combinatorics
using Statistics
using StatsPlots
using RDatasets
using HypothesisTests

R"library(TDA)"

In [ ]:
#load the adjacency matrix
adjacencyMatrix = CSV.read("fullGraphAdjacencyMatrix.csv", copycols = true, header = false)
BirthTimes = CSV.read("From Gene Expression File//279birthTimes.csv", copycols = true, header = false)

In [ ]:
#get a matrix of birth times and sort it
birthTimes = zeros(279)
for i in 1:279
    birthTimes[i] = (BirthTimes[1])[i]
end

sort!(birthTimes)

In [ ]:
#construct a matrix storing all connected pairs (there are 2287 connections)
connectionsList = zeros(2287,2)

counter = 0
for i in 2:279
    for j in 1:i-1
        if adjacencyMatrix[i,j] < 300
            counter += 1
            connectionsList[counter,1] = i
            connectionsList[counter,2] = j
        end
    end
end

In [ ]:
#make a null model replica where we perform 22870 pairwise swaps to generate a random graph with equal degree distribution
ncopies = 100

#make ncopies slots to place copies of adjacency matrix, and pre-load with 300s
nullModel = zeros(279,279,ncopies)
for k in 1:ncopies
    for i in 1:279
        for j in 1:279
            nullModel[i,j,k] = 300
        end
    end
end


for k in 1:ncopies
        
    #perform 22870 pairwise swaps on the connection list, the wise nullModel[:,:,k] with those
    connectionHolder = connectionsList[:,:]
    for i in 1:22870
        edge1 = trunc(Int,rand(1:2287))
        edge2 = trunc(Int,rand(1:2287))
        
        shadowCache = connectionHolder[edge1,2]
        connectionHolder[edge1,2] = connectionHolder[edge2,1]
        connectionHolder[edge2,1] = connectionHolder[edge1,2]
    end
    
    for i in 1:2287
        initial = trunc(Int,connectionHolder[i,1])
        terminal = trunc(Int,connectionHolder[i,2])
        nullModel[initial,terminal,k] = maximum([initial,terminal])
        nullModel[terminal,initial,k] = maximum([initial,terminal])
    end
end

    
    

In [ ]:
BettiCurves = zeros(279,3,ncopies)

for k in 1:ncopies
    
    println(k)

#computes the betti curves in 0-2 dimensions

eireneMatrix = eirene(nullModel[:,:,k], model = "vr", minrad = 0.5, maxrad = 301, maxdim = 2)

#make stuff to store betti curves

barcodes = barcode(eireneMatrix, dim=0)
bigness = size(barcodes,1)
bettiNumber = 0

#calculate and store dimension 0 betti values
for i in 1:bigness
        if barcodes[i,1] == 1
            bettiNumber +=1
        end
end
    
BettiCurves[1,1,k] = bettiNumber

for i in 2:279
        for j in 1:bigness
            if barcodes[j,1] == i
                bettiNumber += 1
            end
            if barcodes[j,2] == i
                bettiNumber -= 1
            end
        end
        BettiCurves[i,1,k] = bettiNumber
end

#calculate and store dimension 1 betti values (resetting variables first)
barcodes = barcode(eireneMatrix, dim=1)
bigness = size(barcodes,1)
bettiNumber = 0

for i in 1:bigness
        if barcodes[i,1] == 1
            bettiNumber +=1
        end
end
    
BettiCurves[1,2,k] = bettiNumber

for i in 2:279
        for j in 1:bigness
            if barcodes[j,1] == i
                bettiNumber += 1
            end
            if barcodes[j,2] == i
                bettiNumber -= 1
            end
        end
        BettiCurves[i,2,k] = bettiNumber
end

#calculate and store dimension 2 betti values (resetting variables first)
barcodes = barcode(eireneMatrix, dim=2)
bigness = size(barcodes,1)
bettiNumber = 0

for i in 1:bigness
        if barcodes[i,1] == 1
            bettiNumber +=1
        end
end
    
BettiCurves[1,3,k] = bettiNumber

for i in 2:279
        for j in 1:bigness
            if barcodes[j,1] == i
                bettiNumber += 1
            end
            if barcodes[j,2] == i
                bettiNumber -= 1
            end
        end
        BettiCurves[i,3,k] = bettiNumber
end
end

In [ ]:
#computes average and standard deviation of 1000 copies
bettiMeans = zeros(279,3)
bettiStd = zeros(279,3)

for i in 1:279
    bettiMeans[i,1] = mean(BettiCurves[i,1,:])
    bettiMeans[i,2] = mean(BettiCurves[i,2,:])
    bettiMeans[i,3] = mean(BettiCurves[i,3,:])
    
    bettiStd[i,1] = std(BettiCurves[i,1,:])
    bettiStd[i,2] = std(BettiCurves[i,2,:])
    bettiStd[i,3] = std(BettiCurves[i,3,:])
end

In [ ]:
plot(birthTimes[:], bettiMeans[:,2:3], ribbon = bettiStd[:,2:3], label = ["dim 1", "dim 2"], title = "Degree Null Model", xlabel = "time", ylabel = "betti number")

In [ ]:
plot(1:279, bettiMeans[:,2:3], ribbon = bettiStd[:,2:3], label = ["dim 1", "dim 2"], title = "Degree Null Model", xlabel = "node", ylabel = "betti number")

In [ ]:
#find the average birth/death rates of the null models



#construct rates matrix for each dimension using a +/- 5 window. column 1 is birth rate, column 2 is death rate
dimensionOneRates = zeros(279,2,ncopies)
dimensionTwoRates = zeros(279,2,ncopies)

for k in 1:ncopies
    println(k)

#find birth and death rate graphs of cavities in dimension 1 and 2 for the kth graph
eireneGraph = eirene(nullModel[:,:,k], model = "vr", minrad = 0.5, maxrad = 301, maxdim = 2)
barcodesOne = barcode(eireneGraph, dim =1)
barcodesTwo = barcode(eireneGraph, dim = 2)


for i in 1:279
    #make and clear counters for new barcodes born and dying in each i+/-5 window
    newBirths1 = 0
    newDeaths1 = 0
    newBirths2 = 0
    newDeaths2 = 0
    
    #look through each barcode to see if one either is born or dies at time i+/1 5
    for j in i-5:i+1
        
        #handle case where j<1
        if j > 0
            newBirths1 += size(findall(x -> x == j, barcodesOne[:,1]))[1]
            newDeaths1 += size(findall(x -> x == j, barcodesOne[:,2]))[1]
        end
        
        if j > 0
            newBirths2 += size(findall(x -> x == j, barcodesTwo[:,1]))[1]
            newDeaths2 += size(findall(x -> x == j, barcodesTwo[:,2]))[1]
        end
    end
        
        
    #set rates at i to be the number of births/deaths in the window centered at i divided by the size of that window
        
    #handle case where i is too close to boundary and the width is not quite 11
    if i <= 5 
        minimum = 1
    end
    if i > 5
        minimum = i-5
    end
    
    if i <= 274 
        maximum = i+5
    end
    if i > 274
        maximum = 279
    end
    
    norm = maximum - minimum + 1
        
    dimensionOneRates[i,1,k] = newBirths1/norm
    dimensionOneRates[i,2,k] = newDeaths1/norm
    dimensionTwoRates[i,1,k] = newBirths2/norm
    dimensionTwoRates[i,2,k] = newDeaths2/norm
    
end
end
println("output: dimensionOneRates and dimensionTwoRates")

In [ ]:
#computes average and standard deviation of birth/death rates
dimOneRateAve = zeros(279,2)
dimOneRateStd = zeros(279,2)
dimTwoRateAve = zeros(279,2)
dimTwoRateStd = zeros(279,2)

for i in 1:279
    dimOneRateAve[i,1] = mean(dimensionOneRates[i,1,:])
    dimOneRateAve[i,2] = mean(dimensionOneRates[i,2,:])
    dimTwoRateAve[i,1] = mean(dimensionTwoRates[i,1,:])
    dimTwoRateAve[i,2] = mean(dimensionTwoRates[i,2,:])
    
    
    dimOneRateStd[i,1] = std(dimensionOneRates[i,1,:])
    dimOneRateStd[i,2] = std(dimensionOneRates[i,2,:])
    dimTwoRateStd[i,1] = std(dimensionTwoRates[i,1,:])
    dimTwoRateStd[i,2] = std(dimensionTwoRates[i,2,:])
end

In [ ]:
dimensionOneRates 

In [ ]:
#plot the one dimension birth and death rates by number of neurons born
p1 = plot(1:279, dimOneRateAve[:,1], ribbon = dimOneRateStd[:,1], title = "(Deg Null) One Dimension Birth Rates in +/- 5 window", legend = false, label = ["birth rate"], xlabel = "number of neurons born", ylabel = "birth rate")
p2 = plot(1:279, dimOneRateAve[:,2], ribbon = dimOneRateStd[:,2], title = "(Deg Null) One Dimension Death Rates in +/- 5 window", legend = false, label = ["death rate"], xlabel = "number of neurons born", ylabel = "death rate")

plot(p1, p2, layout = (2,1))

In [ ]:
#plot the one dimension birth and death rates by time
p1 = plot(birthTimes[:], dimOneRateAve[:,1], ribbon = dimOneRateStd[:,1], title = "(Deg Null) One Dimension Birth Rates in +/- 5 window", legend = false, label = ["birth rate"], xlabel = "time", ylabel = "birth rate")
p2 = plot(birthTimes[:], dimOneRateAve[:,2], ribbon = dimOneRateStd[:,2], title = "(Deg Null) One Dimension Death Rates in +/- 5 window", legend = false, label = ["death rate"], xlabel = "time", ylabel = "death rate")

plot(p1, p2, layout = (2,1))

In [ ]:
#plot the one dimension birth and death rates by number of neurons born
p1 = plot(1:279, dimTwoRateAve[:,1], ribbon = dimTwoRateStd[:,1], title = "(Deg Null) Two Dimension Birth Rates in +/- 5 window", legend = false, label = ["birth rate"], xlabel = "number of neurons born", ylabel = "birth rate")
p2 = plot(1:279, dimTwoRateAve[:,2], ribbon = dimTwoRateStd[:,2], title = "(Deg Null) Two Dimension Death Rates in +/- 5 window", legend = false, label = ["death rate"], xlabel = "number of neurons born", ylabel = "death rate")

plot(p1, p2, layout = (2,1))

In [ ]:
#plot the one dimension birth and death rates by time
p1 = plot(birthTimes[:], dimTwoRateAve[:,1], ribbon = dimTwoRateStd[:,1], title = "(Deg Null) Two Dimension Birth Rates in +/- 5 window", legend = false, label = ["birth rate"], xlabel = "time", ylabel = "birth rate")
p2 = plot(birthTimes[:], dimTwoRateAve[:,2], ribbon = dimTwoRateStd[:,2], title = "(Deg Null) Two Dimension Death Rates in +/- 5 window", legend = false, label = ["death rate"], xlabel = "time", ylabel = "death rate")

plot(p1, p2, layout = (2,1))

In [ ]:
topologicalSimilarityOne = zeros(279,279,10)


In [ ]:
#computes topological similarity for first  null model

k=1
    println("************************************")
#hold barcodes for topological similarity calculation
barcodes = Array{Any, 2}(UndefInitializer(), 279,279)

eireneGraph = eirene(nullModel[:,:,k], minrad = 0.5, maxrad = 301, maxdim = 1)

oneDimBarcodes = barcode(eireneGraph, dim =1)

originalBarcodes = zeros(size(oneDimBarcodes)[1],3)

if size(oneDimBarcodes)[1] > 0
    for i in 1:size(oneDimBarcodes)[1]
        originalBarcodes[i,1] = 1
        originalBarcodes[i,2] = oneDimBarcodes[i,1]
        originalBarcodes[i,3] = oneDimBarcodes[i,2]
    end
end

barcodes[1,1] = originalBarcodes

for i in 2:279
    
    println(i)
    
    for j in 1: i-1
        
        newArray = nullModel[:,:,k]
        
        for a in 1:279
            if newArray[i,a] < 300
                newArray[i,a] = maximum([j,a])
                newArray[a,i] = maximum([j,a])
            end
            
            if newArray[j,a] < 300
                newArray[j,a] = maximum([i,a])
                newArray[a,j] = maximum([i,a])
            end
        end
        
        eireneReordered = eirene(newArray, minrad = 0.5, maxrad = 300, maxdim =1)
        reorderedBarcodes = barcode(eireneReordered, dim =1)
            
        newBarcodes = zeros(size(reorderedBarcodes)[1],3)

        if size(reorderedBarcodes)[1] > 0
            for i in 1:size(reorderedBarcodes)[1]
                newBarcodes[i,1] = 1
                newBarcodes[i,2] = reorderedBarcodes[i,1]
                newBarcodes[i,3] = reorderedBarcodes[i,2]
            end
        end
        
        barcodes[i,j] = newBarcodes
    
    end
end
        

#compute topological similarity for this graph


first = barcodes[1,1]

for i in 2:279
    println(i)
    
    for j in 1: i -1
        
        second = barcodes[i,j]
        
        bottleneckDistance = rcopy(R"bottleneck($first, $second, dimension = 1)")
        
        oneDimensionSimilarity = 1 - ( bottleneckDistance / abs(i-j) )
        
        topologicalSimilarityOne[i,j,k] = oneDimensionSimilarity
        
    end
end

end
println("output: topologicalSimilarityOne")
